In [2]:
import os

os.chdir("../")

In [1]:
%pwd

'c:\\Users\\rupan\\Documents\\Projects\\Deep Learning\\Chicken-Disease-Classification\\research'

In [30]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [31]:

from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [32]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [33]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [34]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [35]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [22]:
# try:
#     config = ConfigurationManager()
#     prepare_callbacks_config = config.get_prepare_callback_config()
#     prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
#     callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

#     training_config = config.get_training_config()
#     training = Training(config=training_config)
#     training.get_base_model()
#     training.train_valid_generator()
#     training.train(
#         callback_list=callback_list
#     )
    
# except Exception as e:
#     raise e

In [36]:
config = ConfigurationManager()
prepare_callbacks_config = config.get_prepare_callback_config()
prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
callback_list = prepare_callbacks.get_tb_ckpt_callbacks()


[2024-06-29 18:12:38,721: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-29 18:12:38,730: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-29 18:12:38,734: INFO: common: created directory at: artifacts]
[2024-06-29 18:12:38,737: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-06-29 18:12:38,740: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [37]:
training_config = config.get_training_config()
training = Training(config=training_config)
training.get_base_model()
training.train_valid_generator()

   

[2024-06-29 18:12:39,742: INFO: common: created directory at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


In [38]:
config=training_config

datagenerator_kwargs = dict(
    rescale = 1./255,
    validation_split=0.20
)

dataflow_kwargs = dict(
    target_size=config.params_image_size[:-1],
    batch_size=config.params_batch_size,
    interpolation="bilinear"
)

valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagenerator_kwargs
)

valid_generator = valid_datagenerator.flow_from_directory(
    directory=config.training_data,
    subset="validation",
    shuffle=False,
    **dataflow_kwargs
)

if config.params_is_augmentation:
    train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=40,
        horizontal_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        **datagenerator_kwargs
    )
else:
    train_datagenerator = valid_datagenerator

train_generator = train_datagenerator.flow_from_directory(
    directory=config.training_data,
    subset="training",
    shuffle=True,
    **dataflow_kwargs
)

Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


In [39]:
(config.training_data)

WindowsPath('artifacts/data_ingestion/Chicken-fecal-images')

In [40]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size

model = tf.keras.models.load_model(config.updated_base_model_path)

In [41]:


model.fit(
train_generator,
epochs=config.params_epochs,
steps_per_epoch=steps_per_epoch,
validation_steps=validation_steps,
validation_data=valid_generator,
callbacks=callback_list
)


# model.save(config.trained_model_path)



c:\Users\rupan\Documents\Projects\Deep Learning\Chicken-Disease-Classification\chicken_venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


KeyboardInterrupt: 

In [17]:
for i in train_generator[0]:
    print(i.shape)

(16, 224, 224, 3)
(16, 3)


In [19]:
batch_data, batch_labels = train_generator[0]  # Get the first batch
print(f"Batch data shape: {batch_data.shape}")
print(f"Batch labels shape: {batch_labels.shape}")
print("First batch data:", batch_data)
print("First batch labels:", batch_labels)


Batch data shape: (16, 224, 224, 3)
Batch labels shape: (16, 3)
First batch data: [[[[0.6188855  0.6188855  0.6188855 ]
   [0.6221248  0.6195839  0.61450213]
   [0.6244925  0.62057096 0.6127278 ]
   ...
   [0.4901961  0.49803925 0.48627454]
   [0.48747903 0.49532217 0.48355746]
   [0.4816017  0.48944482 0.47768012]]

  [[0.633468   0.633468   0.633468  ]
   [0.653616   0.65268403 0.6508202 ]
   [0.6288782  0.6244885  0.61804974]
   ...
   [0.4840046  0.49184772 0.48008302]
   [0.47812724 0.48597038 0.47420567]
   [0.4722499  0.48009303 0.46832833]]

  [[0.64477515 0.64477515 0.64477515]
   [0.67625976 0.67625976 0.67625976]
   [0.63577485 0.62976193 0.6320953 ]
   ...
   [0.4746528  0.48249593 0.47073123]
   [0.46877542 0.47661856 0.46485385]
   [0.46541047 0.4732536  0.4614889 ]]

  ...

  [[0.5883281  0.6275438  0.6628379 ]
   [0.57846    0.6157205  0.6510147 ]
   [0.5686644  0.6039658  0.63925993]
   ...
   [0.66924083 0.6784314  0.6758573 ]
   [0.6616046  0.67729086 0.6812124 ]
   